### **Packages and dataset load**

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sb
from scipy import stats
from scipy.stats import pearsonr
from tqdm import tqdm
# import dataframe_image as df___i

color = {"granate":"#BA4A00",
         "amarillo":"#F5B041",
         "verde":"#148F77",
         "blue":"#0051A2",
         "red": "#DD1717"}
color_palette = [color["blue"], 'darkorchid', color['verde'], color['amarillo'],'gray', 'cornflowerblue', color['red']]
sb.set_style('white')

In [3]:
df = pd.read_csv('Data/Dry_Bean_Dataset.csv')

### **Auxiliar functions**

Cross validation

In [4]:
def dataset_creator(models_names: list, columns_names: list, k1: int):
    header = pd.MultiIndex.from_product([models_names, columns_names])
    df = pd.DataFrame(columns=header)
    df['KFold'] = np.arange(1, k1+1)
    df.set_index('KFold', inplace=True)
    return df

def twolevelcv(X: np.array, y: np.array, k1: int, k2: int, models: list, params: dict, rs: int):
    """Allows to compute two level crossvalidation.

    Args:
        X (np.array): Features (numeric)
        y (np.array): Class (objective variable)
        k1 (int): Nº of outer folds
        k2 (int): Nº of inner folds
        models (list): List of models for comparison
        params (dict): Dictionary including the set of parameters. In this case we only tune 1 parameter per model.
        rs (int): Random state
    Returns:
        df: Dataframe
    """
    test_error_dict = {}
    k = 0
    names = [type(m).__name__ for m in models]
    col_names = ['Param. Value', 'Error']
    df = dataset_creator(names, col_names, k1)
    kf1 = StratifiedKFold(k1, shuffle = True, random_state=rs)
    # first level split
    for train_idx1, test_idx1 in kf1.split(X, y):
        k += 1
        kf2 = StratifiedKFold(k2, shuffle = True, random_state=rs)
        print(f'Computing KFold {k}/{k1}...')
        # second level split
        for train_idx2, test_idx2 in tqdm(kf2.split(X[train_idx1, :], y[train_idx1]), total = k2):
            X_train = X[train_idx2, :]
            y_train = y[train_idx2]
            X_test = X[test_idx2, :]
            y_test = y[test_idx2]
            for name, model in zip(names, models):
                if name != 'DummyClassifier':
                    pname = list(params[name].keys())[0]
                    error_test = []
                    for p_ in params[name][pname]:
                        pdict = {pname: p_}
                        model = model.set_params(**pdict)
                        # train the model
                        model.fit(X_train, y_train)
                        # evaluate performance
                        pred2_test = model.predict(X_test)
                        error_test.append(np.sum(pred2_test != y_test)/ y_test.shape[0])
                    min_param = params[name][pname][np.argmin(error_test)]
                else:
                    model.fit(X_train, y_train)
                    pred2_test = model.predict(X_test)
                    error_test = np.sum(pred2_test != y_test)/ y_test.shape[0]
                    min_param = np.NaN
                df.loc(axis = 1)[name, 'Error'][k] = np.min(error_test)
                df.loc(axis = 1)[name, 'Param. Value'][k] = min_param
    return df, test_idx1

# **1 - Regression**

In [5]:
columns = df.columns.values
X = df.drop(columns='Class').values
y = df['roundness']


print('· NUMBER OF FEATURES:', X.shape[1])
print('\n· FEATURES:\n', columns[:-1])
print('\n· NUMBER OF DATA POINTS:', X.shape[0])

· NUMBER OF FEATURES: 16

· FEATURES:
 ['Area' 'Perimeter' 'MajorAxisLength' 'MinorAxisLength' 'AspectRation'
 'Eccentricity' 'ConvexArea' 'EquivDiameter' 'Extent' 'Solidity'
 'roundness' 'Compactness' 'ShapeFactor1' 'ShapeFactor2' 'ShapeFactor3'
 'ShapeFactor4']

· NUMBER OF DATA POINTS: 13611


### **Part A. *Linear regression.***

### **Part B. *Other models. Evaluation.***

### **Part A. *Linear regression.***

### **Part B. *Other models. Evaluation.***

#### Generate training data x and label y

In [41]:
label_feature = 'AspectRation'

x = df.drop(columns=['Class', label_feature, 'MajorAxisLength', 'MinorAxisLength', \
                     'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', \
                    'ShapeFactor4']).values
Class = df['Class']
class_label = []
dictionary = {
    'SEKER': 0,
    'BARBUNYA': 1,
    'BOMBAY': 2, 
    'CALI': 3,
    'HOROZ': 4,
    'SIRA': 5,
    'DERMASON': 6
}
for i in Class:
    class_label.append(dictionary[i])

class_label = np.array(class_label)
x = np.array(x)
x = np.insert(x, x.shape[1], class_label, axis=1)
y = df[label_feature]
y = np.array(y)
y = y.reshape((-1,1))
x = x.astype(np.float32)
y = y.astype(np.float32)
print(x.shape)
# X = x
N = x.shape[0]


(13611, 9)


### Standardization

In [42]:
X_standard = x - np.ones((N, 1))*x.mean(0)
X_standard = X_standard*(1/np.std(X_standard,0))

X = X_standard
X = X.astype(np.float32)

#### Linear Regression

In [43]:
# import toolbox_02450
from matplotlib.pyplot import figure, plot, xlabel, ylabel, legend, show
import sklearn.linear_model as lm
import numpy as np
import torch.nn as nn
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Fit ordinary least squares regression model
# model = lm.LinearRegression(fit_intercept=True)
model = lm.Ridge(alpha = 1.0, fit_intercept=True)
model = model.fit(X,y)
# Compute model output:
y_est = model.predict(X)
loss = nn.MSELoss()(torch.from_numpy(y).to(device), torch.from_numpy(y_est).to(device))
print(loss.cpu().detach().numpy())

0.0060303346


In [44]:
class Linear_Regression():
    def __init__(self, weight = 0.0) :
        self.weight = weight
        self.w = None
    
    def fit(self,  X: np.array, y: np.array):
        x0 = np.ones((X.shape[0], 1))
        X = np.concatenate((X,x0),axis=1)
        X = np.matrix(X)
        y = np.matrix(y)
        w = np.matmul(X.T, X)
        w = w + self.weight * np.eye(X.shape[1], X.shape[1])
        w = w.I * (X.T * y)
        self.w = w
        return w
    
    def predict(self, X: np.array):
        x0 = np.ones((X.shape[0], 1))
        X = np.concatenate((X,x0),axis=1)
        X = np.matrix(X)
        y_pred = X * self.w
        return y_pred

model = Linear_Regression(0.0)
w = model.fit(X, y)
y_pred = model.predict(X)

loss = nn.MSELoss()(torch.from_numpy(y).to(device), torch.from_numpy(y_pred).to(device))
print(loss.cpu().detach().numpy())


0.006003649539698801


#### baseline

In [45]:
from sklearn.dummy import DummyRegressor
model = DummyRegressor(strategy="mean")
model.fit(X, y)
y_pred = model.predict(X)
loss = nn.MSELoss()(torch.from_numpy(y).to(device), torch.from_numpy(y_pred).to(device))
print(loss.cpu().detach().numpy())

0.060845792


d:\Users\karl\anaconda3\envs\pytorch2\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([13611])) that is different to the input size (torch.Size([13611, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


### ANN

In [46]:
from matplotlib.pylab import figure, plot, xlabel, ylabel, legend, ylim, show
import sklearn.linear_model as lm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

device = "cuda" if torch.cuda.is_available() else 'cpu'
print("device: ", device)

class Neural_Net(nn.Module):
    def __init__(self, num_input, num_output, num_hidden):
        super(Neural_Net, self).__init__()
        self.Net = nn.Sequential(
            nn.Linear(num_input, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, num_output)
        )
        torch.nn.init.xavier_uniform_(self.Net[0].weight)
        torch.nn.init.xavier_uniform_(self.Net[2].weight)
        torch.nn.init.xavier_uniform_(self.Net[4].weight)
        
    def forward(self, input):
        return self.Net(input)

class ANN():
    def __init__(self, num_input, num_output, num_hidden, max_iters):
        self.num_input = num_input
        self.num_output = num_output
        self.num_hidden = num_hidden
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = Neural_Net(num_input, num_output, num_hidden).to(self.device)
        self.loss_func = torch.nn.MSELoss()
        self.max_iters = max_iters
        self.path_root = 'ANN_model/'
    
    def train(self, X: np.array, y: np.array):
        x_train_ = torch.from_numpy(X).to(self.device)
        y_train_ = torch.from_numpy(y).to(self.device)
        self.model.train()
        optim = torch.optim.Adam(self.model.parameters(), lr=1e-4, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optim, 2e4, gamma=0.1, last_epoch=-1)
        for i in range(self.max_iters):
            output = self.model(x_train_)
            loss = self.loss_func(output, y_train_)
            if loss.cpu().detach().numpy() < 1e-6:
                print("early stop")
                break
            optim.zero_grad()
            loss.backward()
            optim.step()
            if i % 10000 == 0:
                print('steps: {}, loss: {}'.format(i, loss.cpu().detach().numpy()))
            scheduler.step()

    def predict(self, X: np.array):
        self.model.eval()
        x_test = torch.from_numpy(X).to(self.device)
        output = self.model(x_test).cpu().detach().numpy()
        return output
    
    def save_model(self, k1, k2):
        PATH = self.path_root + "ANN_{}_{}_{}.pt".format(k1, k2, self.num_hidden)
        torch.save(self.model.state_dict(), PATH)
    
    def load_model(self, k1, k2):
        PATH = self.path_root + "ANN_{}_{}_{}.pt".format(k1, k2, self.num_hidden)
        self.model = Neural_Net(self.num_input, self.num_output, self.num_hidden).to(self.device)
        self.model.load_state_dict(torch.load(PATH))
        
print(y.shape)


device:  cuda
(13611, 1)


In [47]:
# model = ANN(X.shape[1], 1, 64, 10000)
# model.load_model(0,0)
# model.train(X, y)
# model.save_model(0, 0)
# output = model.predict(X)
# error = torch.nn.MSELoss()(torch.from_numpy(output).to(device), torch.from_numpy(y.reshape((-1))).to(device))
# error = error.cpu().detach().numpy()
# print("ANN Model error: {}".format(error))

### Cross Validation for Regression

In [48]:
from sklearn.model_selection import KFold
params = {}
params['Linear_Regression'] = {'lambda': [1.0, 1e-1, 1e-2, 1e-3]}
params['Baseline'] = [None]
params['ANN'] = {'hidden_layer_sizes': [1, 16, 64]}

def twolevelcv_reg(X: np.array, y: np.array, k1: int, k2: int, rs: int, params: dict):
    loss_func = nn.MSELoss()
    k_1 = 0
    names = [n for n in params.keys()]
    col_names = ['Param. Value', 'Error']
    df = dataset_creator(names, col_names, k1)
    kf1 = KFold(n_splits=k1)
    # first level split
    for train_idx1, test_idx1 in kf1.split(X, y):
        k_1 += 1
        kf2 = KFold(n_splits=k2)
        print(f'Computing KFold {k_1}/{k1}...')
        # second level split
        k_2 = 0
        for train_idx2, test_idx2 in tqdm(kf2.split(X[train_idx1, :], y[train_idx1]), total = k2):
            k_2 += 1
            X_train = X[train_idx2, :]
            y_train = y[train_idx2]
            X_test = X[test_idx2, :]
            y_test = y[test_idx2]
            generalization_error_test_x = X[test_idx1, :]
            generalization_error_test_y = y[test_idx1]
            for name in names:
                if name == 'Linear_Regression':
                    error_test = []
                    for i in params[name]['lambda']:
                        model = lm.Ridge(alpha = i, fit_intercept=True)
                        model = model.fit(X_train, y_train)
                        # Compute model output:
                        y_est = model.predict(X_test)
                        error = nn.MSELoss()(torch.from_numpy(y_test.reshape((-1))).to(device), \
                                            torch.from_numpy(y_est.reshape((-1))).to(device))
                        error = error.cpu().detach().numpy()
                        error_test.append(error)
                    min_param = params[name]['lambda'][np.argmin(error_test)]
                elif name == 'ANN':
                    error_test = []
                    for i in params[name]['hidden_layer_sizes']:
                        ANN_model = ANN(X_train.shape[1], 1, i, int(1e5))
                        ANN_model.load_model(k_1, k_2)
                        ANN_model.train(X_train, y_train)
                        ANN_model.save_model(k_1, k_2)
                        output = ANN_model.predict(X_test)
                        error = loss_func(torch.from_numpy(output).to(device), torch.from_numpy(y_test.reshape((-1))).to(device))
                        error = error.cpu().detach().numpy()
                        error_test.append(error)
                        print("ANN Model error: {}".format(error))
                    min_param = params[name]['hidden_layer_sizes'][np.argmin(error_test)]
                else:
                    model = DummyRegressor(strategy="mean")
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)
                    error = nn.MSELoss()(torch.from_numpy(y_test.reshape((-1))).to(device), \
                        torch.from_numpy(y_pred.reshape((-1))).to(device))
                    error = error.cpu().detach().numpy()
                    error_test = error
                    min_param = np.NaN
                df.loc(axis = 1)[name, 'Error'][k_1] = np.min(error_test)
                df.loc(axis = 1)[name, 'Param. Value'][k_1] = min_param
                
    return df, test_idx1

# Table, test_set_outer = twolevelcv_reg(X, y, 5, 5, 1, params)
# Table.to_csv('Results/Regression.csv')


### Section 2 Generalization error

In [49]:
from sklearn.model_selection import KFold
import joblib
params = {}
params['Linear_Regression'] = {'lambda': [1.0, 1e-1, 1e-2, 1e-3]}
params['Baseline'] = [None]
params['ANN'] = {'hidden_layer_sizes': [1, 16, 64]}

def twolevelcv_reg_generalization_error(X: np.array, y: np.array, k1: int, k2: int, rs: int, params: dict):
    loss_func = nn.MSELoss()
    k_1 = 0
    names = [n for n in params.keys()]
    col_names = ['Param. Value', 'Error', 'Generalization Error']
    df = dataset_creator(names, col_names, k1)
    kf1 = KFold(n_splits=k1)
    # first level split
    for train_idx1, test_idx1 in kf1.split(X, y):
        k_1 += 1
        kf2 = KFold(n_splits=k2)
        print(f'Computing KFold {k_1}/{k1}...')
        # second level split
        k_2 = 0
        for train_idx2, test_idx2 in tqdm(kf2.split(X[train_idx1, :], y[train_idx1]), total = k2):
            k_2 += 1
            X_train = X[train_idx2, :]
            y_train = y[train_idx2]
            X_test = X[test_idx2, :]
            y_test = y[test_idx2]
            generalization_error_test_x = X[test_idx1, :]
            generalization_error_test_y = y[test_idx1]
            for name in names:
                if name == 'Linear_Regression':
                    error_test = []
                    gen_error_test = []
                    for i in params[name]['lambda']:
                        model = lm.Ridge(alpha = i, fit_intercept=True)
                        model = model.fit(X_train, y_train)
                        joblib.dump(model, "Linear_Regression/LR_{}_{}_{}.pkl".format(k_1, k_2, i))
                        # Compute model output:
                        y_est = model.predict(X_test)
                        error = nn.MSELoss()(torch.from_numpy(y_test.reshape((-1))).to(device), \
                                            torch.from_numpy(y_est.reshape((-1))).to(device))
                        error = error.cpu().detach().numpy()
                        error_test.append(error)
                        gen_test_y = model.predict(generalization_error_test_x)
                        gen_error = nn.MSELoss()(torch.from_numpy(generalization_error_test_y.reshape((-1))).to(device), \
                                            torch.from_numpy(gen_test_y.reshape((-1))).to(device))
                        gen_error = gen_error.cpu().detach().numpy()
                        gen_error_test.append(gen_error)
                    min_param = params[name]['lambda'][np.argmin(error_test)]
                    gen_error_out = gen_error_test[np.argmin(error_test)]
                    error_out = error_test[np.argmin(error_test)]
                elif name == 'ANN':
                    error_test = []
                    gen_error_test = []
                    for i in params[name]['hidden_layer_sizes']:
                        ANN_model = ANN(X_train.shape[1], 1, i, int(1e5))
                        ANN_model.load_model(k_1, k_2)
                        # ANN_model.train(X_train, y_train)
                        # ANN_model.save_model(k_1, k_2)
                        output = ANN_model.predict(X_test)
                        error = loss_func(torch.from_numpy(output).to(device), torch.from_numpy(y_test.reshape((-1))).to(device))
                        error = error.cpu().detach().numpy()
                        error_test.append(error)
                        gen_test_y = model.predict(generalization_error_test_x)
                        gen_error = nn.MSELoss()(torch.from_numpy(generalization_error_test_y.reshape((-1))).to(device), \
                                            torch.from_numpy(gen_test_y.reshape((-1))).to(device))
                        gen_error = gen_error.cpu().detach().numpy()
                        gen_error_test.append(gen_error)
                    min_param = params[name]['hidden_layer_sizes'][np.argmin(error_test)]
                    gen_error_out = gen_error_test[np.argmin(error_test)]
                    error_out = error_test[np.argmin(error_test)]
                else:
                    model = DummyRegressor(strategy="mean")
                    model.fit(X_train, y_train)
                    joblib.dump(model, "BaseLine/BL_{}_{}.pkl".format(k_1, k_2))
                    y_pred = model.predict(X_test)
                    error = nn.MSELoss()(torch.from_numpy(y_test.reshape((-1))).to(device), \
                        torch.from_numpy(y_pred.reshape((-1))).to(device))
                    error = error.cpu().detach().numpy()
                    error_test = error
                    gen_test_y = model.predict(generalization_error_test_x)
                    gen_error = nn.MSELoss()(torch.from_numpy(generalization_error_test_y.reshape((-1))).to(device), \
                                            torch.from_numpy(gen_test_y.reshape((-1))).to(device))
                    gen_error = gen_error.cpu().detach().numpy()
                    min_param = np.NaN
                    gen_error_out = gen_error_test[np.argmin(error_test)]
                    error_out = error
                df.loc(axis = 1)[name, 'Error'][k_1] = error_out
                df.loc(axis = 1)[name, 'Param. Value'][k_1] = min_param
                df.loc(axis = 1)[name, 'Generalization Error'][k_1] = gen_error_out
                
    return df, test_idx1

# Table, test_set_outer = twolevelcv_reg_generalization_error(X, y, 5, 5, 1, params)
# Table.to_csv('Results/Regression_Generalization_Error.csv')

### Sectin 3 Compare of different models using Setup 1

#### P-value and Confidencial Interval

In [50]:
import scipy.stats as st
mA = joblib.load('Linear_Regression/LR_1_1_0.001.pkl')
mB = joblib.load('BaseLine/BL_1_1.pkl')
yhatA = mA.predict(X)
yhatB = mB.predict(X)
yhatB = np.reshape(yhatB, (-1,1))
def P_value_CI(mA, mB, X, y):
    yhatA = mA.predict(X)
    yhatA = np.reshape(yhatA, (-1,1))
    yhatB = mB.predict(X)
    yhatB = np.reshape(yhatB, (-1,1))
    zA = np.abs(y - yhatA ) ** 2

    # compute confidence interval of model A
    alpha = 0.05
    CIA = st.t.interval(1-alpha, df=len(zA)-1, loc=np.mean(zA), scale=st.sem(zA))  # Confidence interval

    # Compute confidence interval of z = zA-zB and p-value of Null hypothesis
    zB = np.abs(y - yhatB ) ** 2
    z = zA - zB
    CI = st.t.interval(1-alpha, len(z)-1, loc=np.mean(z), scale=st.sem(z))  # Confidence interval
    p = 2*st.t.cdf( -np.abs( np.mean(z) )/st.sem(z), df=len(z)-1)  # p-value
    s, p = st.ttest_rel(yhatA, yhatB)
    return p, CI, s


#### Paired T test

In [60]:
def Models_comparison(X: np.array, y: np.array, k1: int, k2: int, rs: int, params: dict):
    loss_func = nn.MSELoss()
    k_1 = 0
    names = ['LR vs NN', 'LR vs BL', 'NN vs BL']
    col_names = ['t-test', 'p-value', 'Confidenctial Interval']
    df = dataset_creator(names, col_names, k1)
    kf1 = KFold(n_splits=k1)
    # first level split
    for train_idx1, test_idx1 in kf1.split(X, y):
        k_1 += 1
        kf2 = KFold(n_splits=k2)
        print(f'Computing KFold {k_1}/{k1}...')
        # second level split
        k_2 = 0
        for train_idx2, test_idx2 in tqdm(kf2.split(X[train_idx1, :], y[train_idx1]), total = k2):
            k_2 += 1
            X_train = X[train_idx2, :]
            y_train = y[train_idx2]
            X_test = X[test_idx2, :]
            y_test = y[test_idx2]
            generalization_error_test_x = X[test_idx1, :]
            generalization_error_test_y = y[test_idx1]
            model_NN = ANN(X_train.shape[1], 1, 64, int(1e5))
            model_LR = joblib.load('Linear_Regression/LR_{}_{}_0.001.pkl'.format(k_1, k_2))
            model_NN.load_model(k_1, k_2)
            model_BL = joblib.load('BaseLine/BL_{}_{}.pkl'.format(k_1, k_2))
            for name in names:
                if name == 'LR vs NN':
                    p, CI, stats = P_value_CI(model_LR, model_NN, generalization_error_test_x, generalization_error_test_y)
                    df.loc(axis = 1)[name, 't-test'][k_1] = stats
                    df.loc(axis = 1)[name, 'p-value'][k_1] = p
                    df.loc(axis = 1)[name, 'Confidenctial Interval'][k_1] = CI
                elif 'LR vs BL':
                    p, CI, stats = P_value_CI(model_LR, model_BL, generalization_error_test_x, generalization_error_test_y)
                    df.loc(axis = 1)[name, 't-test'][k_1] = stats
                    df.loc(axis = 1)[name, 'p-value'][k_1] = p
                    df.loc(axis = 1)[name, 'Confidenctial Interval'][k_1] = CI
                else:
                    p, CI, stats = P_value_CI(model_NN, model_BL, generalization_error_test_x, generalization_error_test_y)
                    df.loc(axis = 1)[name, 't-test'][k_1] = stats
                    df.loc(axis = 1)[name, 'p-value'][k_1] = p
                    df.loc(axis = 1)[name, 'Confidenctial Interval'][k_1] = CI
            
    return df, test_idx1

Table, test_set_outer = Models_comparison(X, y, 5, 5, 1, params)
Table.to_csv('Results/Regression_P_value.csv')

Computing KFold 1/5...


100%|██████████| 5/5 [00:00<00:00, 38.85it/s]


Computing KFold 2/5...


100%|██████████| 5/5 [00:00<00:00, 52.96it/s]


Computing KFold 3/5...


100%|██████████| 5/5 [00:00<00:00, 57.82it/s]


Computing KFold 4/5...


100%|██████████| 5/5 [00:00<00:00, 49.19it/s]


Computing KFold 5/5...


100%|██████████| 5/5 [00:00<00:00, 68.22it/s]


# **2 - Classification**

### **Dataset preparation**

In [5]:
# Copying object without editing the original
df_ = df.copy(deep=True)
# Doing this we can choose to use outliers filter or not

In [6]:
columns = df_.columns.values
X = df_.drop(columns='Class').values
y = df_['Class']
le = LabelEncoder()
y_ = le.fit_transform(y)
classes = y.unique()

print('· NUMBER OF FEATURES:', X.shape[1])
print('\n· FEATURES:', columns[:-1])
print('\n· NUMBER OF DATA POINTS:', X.shape[0])
print('\n· CLASSES:', classes)
print('\n· NUMBER OF CLASSES:', len(classes))

· NUMBER OF FEATURES: 16

· FEATURES: ['Area' 'Perimeter' 'MajorAxisLength' 'MinorAxisLength' 'AspectRation'
 'Eccentricity' 'ConvexArea' 'EquivDiameter' 'Extent' 'Solidity'
 'roundness' 'Compactness' 'ShapeFactor1' 'ShapeFactor2' 'ShapeFactor3'
 'ShapeFactor4']

· NUMBER OF DATA POINTS: 13611

· CLASSES: ['SEKER' 'BARBUNYA' 'BOMBAY' 'CALI' 'HOROZ' 'SIRA' 'DERMASON']

· NUMBER OF CLASSES: 7


#### **Transformations**

Outliers removal

In [7]:
Threshold_ = 3
outlier_index = []
df_ = pd.DataFrame(columns=df.columns)
index = 0
for K in classes:
    outlier_index = []
    a = df.loc[df["Class"] == K]
    value = a.drop(columns='Class').values
    for j in range(16):
        std = np.std(value[:, j])
        mean = np.mean(value[:, j])
        for i in range(value[:, j].shape[0]):
            if (value[i, j] - mean) / std > Threshold_:
                outlier_index.append(i + index)
    index = i + index + 1
    outlier_index = np.unique(outlier_index)
    a = a.drop(outlier_index)
    df_ = pd.concat([df_,a])
df_.reset_index(drop=True, inplace=True)
print(f'Filtered outliers: {df.shape[0] - df_.shape[0]}')

Filtered outliers: 298


Standarization

In [8]:
# Standarization of the dataset
sc = StandardScaler()
X_stdz = sc.fit_transform(X)
df_stdz = pd.DataFrame(columns = columns[:-1], data = X_stdz)
df_stdz['Class'] = y_

### **2.2 Logistic regression *vs.* Neural Network *vs.* Baseline**

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier 
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
random_state = 1

#### Logistic regression

In [10]:
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [11]:
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate the model and collect the scores
# n_scores = cross_val_score(model, X_stdz, y_, scoring = 'accuracy', cv=cv, n_jobs=-1)
# # report the model performance
# print('Mean Accuracy: %.3f (+-%.3f)' % (np.mean(n_scores), np.std(n_scores)))

---

### **2.3 Cross-Validation table**

In [22]:
params = {}
lam = np.logspace(-6, 2, 100)
C = 1/ lam
# C = [200000000, 10000000, 0.1519911082952933, 0.2848035868435805 ]
params['LogisticRegression'] = {'C': C}
params['DummyClassifier'] = [None]
params['MLPClassifier'] = {'hidden_layer_sizes': [(8, ), (16, ), (20, )]}
models = [LogisticRegression(multi_class='multinomial', solver='saga', max_iter=1000000, random_state= random_state, tol = 0.003, n_jobs = -1),
        DummyClassifier(strategy='most_frequent', random_state=random_state),
        MLPClassifier(solver='adam', activation='logistic', alpha=1e-4, random_state=random_state, max_iter=1000, 
        early_stopping=True, validation_fraction=0.2, warm_start=True, verbose=False, learning_rate ='adaptive', learning_rate_init=0.01)]
k1 = 10
k2 = 10
Table, test_set_outer = twolevelcv(X = X_stdz, y = y_, k1 = k1, k2 = k2, models = models, params = params, rs = random_state)
Table.to_csv('Results/Test2_saga.csv')

Computing KFold 1/10...


100%|██████████| 10/10 [06:47<00:00, 40.74s/it]


Computing KFold 2/10...


100%|██████████| 10/10 [07:13<00:00, 43.32s/it]


Computing KFold 3/10...


...

100%|██████████| 10/10 [12:31<00:00, 75.16s/it]


Computing KFold 9/10...


100%|██████████| 10/10 [10:49<00:00, 64.91s/it]


Computing KFold 10/10...


100%|██████████| 10/10 [11:46<00:00, 70.62s/it]


In [31]:
Table

LogisticRegression           DummyClassifier           MLPClassifier  \
            Param. Value     Error    Param. Value     Error  Param. Value   
KFold                                                                        
1              1000000.0  0.040033             NaN  0.788399         (16,)   
2              1000000.0  0.035102             NaN   0.78449          (8,)   
3              1000000.0  0.035102             NaN   0.78449          (8,)   
4              1000000.0  0.035102             NaN   0.78449          (8,)   
5              1000000.0  0.035102             NaN   0.78449          (8,)   
6               0.722081  0.036735             NaN  0.790204          (8,)   
7               0.722081  0.036735             NaN  0.790204          (8,)   
8               0.236449  0.033469             NaN  0.790204          (8,)   
9               0.236449  0.033469             NaN  0.790204          (8,)   
10              0.284804  0.041633             NaN  0.788571          (8,)   

                 
          Error  
KFold            
1      0.039216  
2      0.035102  
3      0.034286  
4      0.035918  
5      0.035918  
6      0.040816  
7          0.04  
8      0.041633  
9      0.042449  
10     0.044082

In [29]:
Table = Table.round(3)
Table.to_csv(r'Results\Table_classification.csv')

In [34]:
test_set_outer.shape

(1361,)

---

### **2.4 Stadistical Evaluation**

In [96]:
from itertools import combinations

def McNemar(models: list, X: np.array, y: np.array, k1: int, rs: int):
    """Computes the McNemar matrix for all the models in the list 

    Args:
        models (list): list of models
        X (np.array): Features
        y (np.array): Classes
        k1 (int): Number of folds
        rs (int): Random state

    Returns:
        _type_: Matrix as a dictionary
    """
    kf1 = StratifiedKFold(k1, shuffle = True, random_state=rs)
    k = 0
    # setting up all the possible combinations between the different models
    matrix = dict.fromkeys(combinations(range(len(models)), 2))
    matrix_tot = dict.fromkeys(combinations(range(len(models)), 2))
    for train_idx, test_idx in kf1.split(X, y):
        test_size = test_idx.shape[0]
        yABC = np.empty(shape=(len(models), test_size))
        for i, model in enumerate(models):
            model.fit(X[train_idx,:], y[train_idx])
            y_pred = model.predict(X[test_idx, :])
            yABC[i, :] = 1*(y_pred == y[test_idx])
        for j in list(matrix.keys()):
            if k == 0:
                matrix[j] = np.empty(shape=(k1, 4))
            n11 = np.sum(yABC[j[0],:]*yABC[j[1],:])
            n12 = np.sum(yABC[j[0],:]*(1-yABC[j[1],:]))
            n21 = np.sum(yABC[j[1],:]*(1-yABC[j[0],:]))
            n22 = np.sum((1-yABC[0,:])*(1-yABC[1,:]))
            matrix[j][k] = np.array([n11, n12, n21, n22])
            matrix_tot[j] = matrix[j].sum(axis = 0)
        k+=1
    return matrix, matrix_tot

In [97]:
random_state = 1
models = [LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state= random_state, C = 0.07),
        DummyClassifier(strategy='most_frequent', random_state=random_state),
        MLPClassifier(solver='adam', activation='logistic', alpha=1e-4, random_state=random_state, max_iter=1000, hidden_layer_sizes=(8, ),
        early_stopping=True, validation_fraction=0.2, warm_start=True, verbose=False, learning_rate ='adaptive', learning_rate_init=0.01)]
k1 = 10
mfull, m_tot = McNemar(models, X_stdz[test_set_outer, :], y_[test_set_outer], k1, rs = random_state)

In [114]:
for mat in mfull:
    print('          ',mat)
    for ma in mfull[mat]:
        print('      ', ma)
    print('\nTotal:',m_tot[mat])
    print('----------------------------------')

           (0, 1)
       [34. 92.  2.  9.]
       [34. 93.  1.  8.]
       [32. 92.  3.  9.]
       [35. 91.  0. 10.]
       [34. 98.  1.  3.]
       [34. 84.  1. 17.]
       [31. 93.  5.  7.]
       [33. 96.  3.  4.]
       [31. 93.  5.  7.]
       [32. 94.  4.  6.]

Total: [330. 926.  25.  80.]
----------------------------------
           (0, 2)
       [121.   5.   2.   9.]
       [121.   6.   2.   8.]
       [123.   1.   2.   9.]
       [124.   2.   2.  10.]
       [128.   4.   0.   3.]
       [113.   5.   2.  17.]
       [123.   1.   1.   7.]
       [128.   1.   1.   4.]
       [123.   1.   5.   7.]
       [124.   2.   1.   6.]

Total: [1228.   28.   18.   80.]
----------------------------------
           (1, 2)
       [32.  4. 91.  9.]
       [33.  2. 90.  8.]
       [32.  3. 93.  9.]
       [34.  1. 92. 10.]
       [34.  1. 94.  3.]
       [33.  2. 82. 17.]
       [31.  5. 93.  7.]
       [32.  4. 97.  4.]
       [32.  4. 96.  7.]
       [31.  5. 94.  6.]

Total: [324.  31. 922

We create the matrix with shape (3: number of models, 10: nº of k-folds, 4: matrix shape)

The matrix is squeezed so we have:
$$\begin{bmatrix}
 n_{11}    & n_{12}  \\
 n_{21}    & n_{22}
\end{bmatrix}$$

Here is $[ n_{11}, n_{12}, n_{21}, n_{22} ]$

$H_0:$ Model A has the same performarnce as model B 

$H_1:$ Model A and model B has different performance

small p_value-> we discard H0 -> Model A and Model B have different performance

Also we set:

**Model 0**: LR

**Model 1**: Baseline

**Model 2**: MLP


### ***P-Values***

In [106]:
combs = list(combinations(range(len(models)), 2))

In [107]:
from scipy.stats import binom
pv_dict = dict.fromkeys(combs)
for j in combs:
    vals = m[j][:, 1:3]
    pv_dict[j] = [binom.cdf(min(vals[i]), n = sum(vals[i]), p = 1/2) for i in range(len(vals))]

IndexError: invalid index to scalar variable.

In [39]:
McNemar_pv = pd.DataFrame(columns = combs, index = range(10))

In [57]:
i = 0
for n in pv_dict.values():
    print(f'{combs[i]}')
    for j, k in enumerate(n):
        McNemar_pv[combs[i]][j] = "{:.3e}".format(k)
        text = str("${:.2e}".format(k))
        text = text + '}$'
        text = text.replace("e", "·10^{")
        print(text)
    i+=1

(0, 1)
$2.25·10^{-25}$
$4.80·10^{-27}$
$3.61·10^{-24}$
$4.04·10^{-28}$
$1.58·10^{-28}$
$2.22·10^{-24}$
$2.26·10^{-22}$
$2.55·10^{-25}$
$2.26·10^{-22}$
$1.19·10^{-23}$
(0, 2)
$2.27·10^{-01}$
$1.45·10^{-01}$
$5.00·10^{-01}$
$6.88·10^{-01}$
$6.25·10^{-02}$
$2.27·10^{-01}$
$7.50·10^{-01}$
$7.50·10^{-01}$
$1.09·10^{-01}$
$5.00·10^{-01}$
(1, 2)
$8.40·10^{-23}$
$8.64·10^{-25}$
$1.86·10^{-24}$
$9.49·10^{-27}$
$2.42·10^{-27}$
$1.85·10^{-22}$
$2.26·10^{-22}$
$1.68·10^{-24}$
$3.22·10^{-24}$
$1.19·10^{-22}$


In [58]:
McNemar_pv.to_csv('Results/McNemar_pv_best.csv')

### Confidence intervals

In [59]:
from scipy.stats import beta

In [60]:
def calcs(mat: np.array):
    """Calculate f y g from a McNemar Matrix
    Args:
        matrix (np.array): McNemar matrix from one K-fold
    Returns:
        _type_: f and g
    """
    n = mat.sum()
    n12 = mat[1]
    n21 = mat[2]
    E_th = (n12 - n21)/n 
    Q = (n**2 * (n+1) * (E_th +1) * (1-E_th)) / (n * (n12 + n21) - (n12 - n21)**2)
    f = (Q-1)*(E_th+1)/2
    g = (Q-1)*(1-E_th)/2
    return f, g

In [61]:
def interval(mat: np.array, alpha: float):
    """McNemar confidence interval

    Args:
        alpha (float): The desired confidence (should be 0.05)  
        f (_type_): output of calcs
        g (_type_): output of calcs 

    Returns:
        _type_: left and right bounds from the interval
    """
    f,g = calcs(mat)
    theta_L = 2*beta.ppf(alpha, f, g) - 1
    theta_R = 2*beta.ppf(1 - alpha/2, f, g) - 1
    return theta_L, theta_R

In [62]:
for i in m:
    print(i)
    for mat in m[i]:
        theta_L, theta_R = interval(mat, 0.05)
        print(f'[{np.round(theta_L, 2)}, {np.round(theta_R, 2)}]')

(0, 1)
[0.58, 0.74]
[0.61, 0.75]
[0.58, 0.74]
[0.6, 0.74]
[0.64, 0.79]
[0.54, 0.69]
[0.57, 0.73]
[0.61, 0.76]
[0.57, 0.73]
[0.58, 0.75]
(0, 2)
[-0.01, 0.06]
[-0.0, 0.07]
[-0.03, 0.02]
[-0.02, 0.03]
[0.01, 0.06]
[-0.01, 0.06]
[-0.02, 0.02]
[-0.02, 0.02]
[-0.06, 0.01]
[-0.01, 0.03]
(1, 2)
[-0.71, -0.55]
[-0.73, -0.57]
[-0.73, -0.57]
[-0.73, -0.58]
[-0.77, -0.62]
[-0.67, -0.51]
[-0.72, -0.55]
[-0.75, -0.59]
[-0.73, -0.57]
[-0.73, -0.56]


### **2.5 Train logistic regression model**

So in the fourth exercise do we have to repeat the parameter selection process or can just go ahead with the best parameter selection for each model?